# 2. Exploring the relationship between gender and policing

Apakah jenis kelamin pengemudi berdampak pada perilaku polisi selama lalu lintas berhenti? Dalam bab ini, Anda akan menjelajahi pertanyaan itu sambil mempraktekkan filtering, grouping, method chaining, Boolean math, string methods, dan banyak lagi!

## Preparing Data

In [2]:
# Import the pandas library as pd
import pandas as pd

# Assign url file
file_police = 'https://assets.datacamp.com/production/repositories/1497/datasets/62bd9feef451860db02d26553613a299721882e8/police.csv'

# Read 'police.csv' into a DataFrame named ri
ri = pd.read_csv(file_police)

# Drop the 'county_name' and 'state' columns
ri.drop(['county_name', 'state'], axis='columns', inplace=True)

# Drop all rows that are missing 'driver_gender'
ri.dropna(subset=['driver_gender'], inplace=True)

# Change the data type of 'is_arrested' to 'bool'
ri['is_arrested'] = ri.is_arrested.astype('bool')

# Concatenate 'stop_date' and 'stop_time' (separated by a space)
combined = ri.stop_date.str.cat(ri.stop_time, sep=' ')

# Convert 'combined' to datetime format
ri['stop_datetime'] = pd.to_datetime(combined)

# Set 'stop_datetime' as the index
ri.set_index('stop_datetime', inplace=True)

## Do the genders commit different violations?

### Examining traffic violations

Sebelum membandingkan pelanggaran yang dilakukan oleh masing-masing jenis kelamin, Anda harus memeriksa pelanggaran yang dilakukan oleh semua pengemudi untuk mendapatkan pemahaman dasar tentang data tersebut.

Dalam latihan ini, Anda akan menghitung nilai unik di kolom `violation`, dan lalu secara terpisah menyatakan jumlah tersebut sebagai proporsi.

In [3]:
# Count the unique values in 'violation'
print(ri.violation.value_counts())

# Express the counts as proportions
print(ri.violation.value_counts(normalize=True))

Speeding               48423
Moving violation       16224
Equipment              10921
Other                   4409
Registration/plates     3703
Seat belt               2856
Name: violation, dtype: int64
Speeding               0.559571
Moving violation       0.187483
Equipment              0.126202
Other                  0.050950
Registration/plates    0.042791
Seat belt              0.033004
Name: violation, dtype: float64


**Note** : Menarik! Lebih dari setengah dari semua pelanggaran adalah melampaui batas kecepatan, diikuti oleh pelanggaran bergerak lainnya dan pelanggaran perlengkapan.

### Comparing violations by gender

Pertanyaan yang kita coba jawab adalah apakah pengemudi pria dan wanita cenderung melakukan berbagai jenis pelanggaran lalu lintas.

Dalam latihan ini, pertama-tama Anda akan membuat DataFrame untuk setiap gender, dan kemudian menganalisis pelanggaran di setiap DataFrame secara terpisah.

In [4]:
# Create a DataFrame of female drivers
female = ri[ri.driver_gender == 'F']

# Create a DataFrame of male drivers
male = ri[ri.driver_gender == 'M']

# Compute the violations by female drivers (as proportions)
print(female.violation.value_counts(normalize=True))

# Compute the violations by male drivers (as proportions)
print(male.violation.value_counts(normalize=True))

Speeding               0.658114
Moving violation       0.138218
Equipment              0.105199
Registration/plates    0.044418
Other                  0.029738
Seat belt              0.024312
Name: violation, dtype: float64
Speeding               0.522243
Moving violation       0.206144
Equipment              0.134158
Other                  0.058985
Registration/plates    0.042175
Seat belt              0.036296
Name: violation, dtype: float64


**Note** : Sekitar dua pertiga dari perhentian lalu lintas perempuan melanggar batas kecepatan, sedangkan pemberhentian laki-laki lebih seimbang di antara enam kategori. Namun, ini tidak berarti bahwa wanita lebih cepat berpacu daripada pria, karena kami tidak memperhitungkan jumlah pemberhentian atau driver.

## Does gender affect who gets a ticket for speeding?

### Filtering by multiple conditions

Memfilter dataFrame `ri` untuk hanya menyertakan driver wanita yang dihentikan karena pelanggaran kecepatan.

<pre>
ri[(ri.driver_gender == 'F') & (ri.violation == 'Speeding')]
</pre>

### Comparing speeding outcomes by gender

Ketika seorang pengemudi menepi karena ngebut, banyak orang percaya bahwa gender berdampak pada apakah pengemudi akan menerima tiket atau peringatan. Bisakah Anda menemukan bukti ini di dataset?

Pertama, Anda akan membuat dua DataFrames driver yang dihentikan karena ngebut: satu berisi perempuan dan laki-laki lainnya berisi.

Kemudian, untuk setiap jenis kelamin, Anda akan menggunakan kolom stop_outcome untuk menghitung berapa persentase pemberhentian yang menghasilkan "Citation" (artinya tiket) versus "Warning".

In [5]:
# Create a DataFrame of female drivers stopped for speeding
female_and_speeding = ri[(ri.driver_gender == 'F') & (ri.violation == 'Speeding')]

# Create a DataFrame of male drivers stopped for speeding
male_and_speeding = ri[(ri.driver_gender == 'M') & (ri.violation == 'Speeding')]

# Compute the stop outcomes for female drivers (as proportions)
print(female_and_speeding.stop_outcome.value_counts(normalize=True))

# Compute the stop outcomes for male drivers (as proportions)
print(male_and_speeding.stop_outcome.value_counts(normalize=True))

Citation            0.952192
Warning             0.040074
Arrest Driver       0.005752
N/D                 0.000959
Arrest Passenger    0.000639
No Action           0.000383
Name: stop_outcome, dtype: float64
Citation            0.944595
Warning             0.036184
Arrest Driver       0.015895
Arrest Passenger    0.001281
No Action           0.001068
N/D                 0.000976
Name: stop_outcome, dtype: float64


**Note** : Menarik! Jumlahnya sama untuk pria dan wanita: sekitar 95% dari perhentian karena ngebut hasilnya tiket. Dengan demikian, data gagal menunjukkan bahwa gender berdampak pada siapa yang mendapatkan tiket karena ngebut.

## Does gender affect whose vehicle is searched?

### Calculating the search rate

Selama pemberhentian lalu lintas, petugas polisi terkadang melakukan pencarian kendaraan. Dalam latihan ini, Anda akan menghitung persentase semua perhentian di DataFrame `ri` yang menghasilkan pencarian kendaraan, juga dikenal sebagai tingkat pencarian.

In [6]:
# Check the data type of 'search_conducted'
print(ri.search_conducted.dtype)

# Calculate the search rate by counting the values
print(ri.search_conducted.value_counts(normalize=True))

# Calculate the search rate by taking the mean
print(ri.search_conducted.mean())

bool
False    0.961785
True     0.038215
Name: search_conducted, dtype: float64
0.0382153092354627


**Note** : Sepertinya tingkat pencarian sekitar 3,8%. Selanjutnya, Anda akan memeriksa apakah tingkat pencarian bervariasi berdasarkan jenis kelamin pengemudi.

### Comparing search rates by gender

Dalam latihan ini, Anda akan membandingkan angka pencarian pengemudi wanita dan pria selama pemberhentian lalu lintas. Ingat bahwa tingkat pencarian kendaraan di semua halte adalah sekitar 3,8%.

Pertama, Anda akan memfilter DataFrame berdasarkan gender dan menghitung tingkat pencarian untuk setiap grup secara terpisah. Kemudian, Anda akan melakukan perhitungan yang sama untuk kedua jenis kelamin sekaligus menggunakan `.groupby()`.

In [7]:
# Calculate the search rate for female drivers
print(ri[ri.driver_gender == 'F'].search_conducted.mean())

# Calculate the search rate for male drivers
print(ri[ri.driver_gender == 'M'].search_conducted.mean())

# Calculate the search rate for each combination of gender and violation
print(ri.groupby('driver_gender').search_conducted.mean())

0.019180617481282074
0.04542557598546892
driver_gender
F    0.019181
M    0.045426
Name: search_conducted, dtype: float64


### Adding a second factor to the analysis

Meskipun tingkat pencarian untuk pria jauh lebih tinggi daripada wanita, mungkin saja perbedaannya sebagian besar disebabkan oleh faktor kedua.

Misalnya, Anda mungkin berhipotesis bahwa tingkat pencarian bervariasi berdasarkan jenis pelanggaran, dan perbedaan dalam tingkat pencarian antara pria dan wanita adalah karena mereka cenderung melakukan pelanggaran yang berbeda.

Anda dapat menguji hipotesis ini dengan memeriksa tingkat pencarian untuk setiap kombinasi gender dan pelanggaran. Jika hipotesis itu benar, Anda akan menemukan bahwa pria dan wanita dicari pada tingkat yang sama untuk setiap pelanggaran. Cari tahu di bawah ini jika itu masalahnya!

In [8]:
# Calculate the search rate for each combination of gender and violation
print(ri.groupby(['driver_gender', 'violation']).search_conducted.mean())

driver_gender  violation          
F              Equipment              0.039984
               Moving violation       0.039257
               Other                  0.041018
               Registration/plates    0.054924
               Seat belt              0.017301
               Speeding               0.008309
M              Equipment              0.071496
               Moving violation       0.061524
               Other                  0.046191
               Registration/plates    0.108802
               Seat belt              0.035119
               Speeding               0.027885
Name: search_conducted, dtype: float64


In [9]:
# Reverse the ordering to group by violation before gender
print(ri.groupby(['violation', 'driver_gender']).search_conducted.mean())

violation            driver_gender
Equipment            F                0.039984
                     M                0.071496
Moving violation     F                0.039257
                     M                0.061524
Other                F                0.041018
                     M                0.046191
Registration/plates  F                0.054924
                     M                0.108802
Seat belt            F                0.017301
                     M                0.035119
Speeding             F                0.008309
                     M                0.027885
Name: search_conducted, dtype: float64


**Note** : Untuk semua jenis pelanggaran, tingkat pencarian lebih tinggi untuk pria daripada wanita, menyangkal hipotesis kami.

## Does gender affect who is frisked during a search?

### Counting protective frisks

Selama pencarian kendaraan, petugas polisi dapat menepuk pengemudi untuk memeriksa apakah mereka memiliki senjata. Ini dikenal sebagai "protective frisk."

Dalam latihan ini, pertama-tama Anda akan memeriksa untuk melihat berapa kali "Protective Frisk" adalah satu-satunya jenis pencarian. Kemudian, Anda akan menggunakan metode string untuk menemukan semua contoh di mana driver itu digeledah.

In [10]:
# Count the 'search_type' values
print(ri.search_type.value_counts())

# Check if 'search_type' contains the string 'Protective Frisk'
ri['frisk'] = ri.search_type.str.contains('Protective Frisk', na=False)

# Check the data type of 'frisk'
print(ri.frisk.dtype)

# Take the sum of 'frisk'
print(ri.frisk.sum())

Incident to Arrest                                          1290
Probable Cause                                               924
Inventory                                                    219
Reasonable Suspicion                                         214
Protective Frisk                                             164
Incident to Arrest,Inventory                                 123
Incident to Arrest,Probable Cause                            100
Probable Cause,Reasonable Suspicion                           54
Incident to Arrest,Inventory,Probable Cause                   35
Probable Cause,Protective Frisk                               35
Incident to Arrest,Protective Frisk                           33
Inventory,Probable Cause                                      25
Protective Frisk,Reasonable Suspicion                         19
Incident to Arrest,Inventory,Protective Frisk                 18
Incident to Arrest,Probable Cause,Protective Frisk            13
Inventory,Protective Fris

**Note** : Sepertinya ada 303 driver yang digeledah. Selanjutnya, Anda akan memeriksa apakah jenis kelamin memengaruhi siapa yang digeledah.

### Comparing frisk rates by gender

Dalam latihan ini, Anda akan membandingkan angka di mana pengemudi wanita dan pria digeledah selama pencarian. Apakah laki-laki digeledah lebih sering daripada perempuan, mungkin karena petugas polisi menganggap mereka berisiko lebih tinggi?

Sebelum melakukan perhitungan apa pun, penting untuk memfilter DataFrame hanya menyertakan subset data yang relevan, yaitu berhenti di mana pencarian dilakukan (`search_conducted`).

In [11]:
# Create a DataFrame of stops in which a search was conducted
searched = ri[ri.search_conducted == True]

# Calculate the overall frisk rate by taking the mean of 'frisk'
print(searched.frisk.mean())

# Calculate the frisk rate for each gender
print(searched.groupby('driver_gender').frisk.mean())

0.09162382824312065
driver_gender
F    0.074561
M    0.094353
Name: frisk, dtype: float64


**Note** : Tingkat frisk lebih tinggi untuk pria daripada wanita, meskipun kami tidak dapat menyimpulkan bahwa perbedaan ini disebabkan oleh jenis kelamin pengemudi.